In [57]:
# Import Libraries
import pandas as pd
import numpy as np
import xlsxwriter
import matplotlib.pyplot as plt

#show plot in notebook
%matplotlib inline 

#limits amount of lines pandas prints out
from pandas import set_option 
set_option('display.max_rows', 10)

# Importing and Cleaning Data

In [58]:
#Loading the main dataset from excel into python, specified taking data from the 'Sales' sheet
Sales_df = pd.read_excel('Sales Data.xlsm', sheetname = 'Sales')
Sales_df

,Exchange Rate,Customer,Field,Date of Order,Total Sales,Year of First Order
0,1.2563,3M Maplewood,solutions provider,2015-04-13,3267.00,2009
1,1.2188,ABB Australia Pty Limited,engine technology,2015-05-21,480.00,2008
2,1.1800,Accutech Instrumentation,distributor,2015-01-13,18990.40,2013
3,1.2524,Advanced Analytical Technologies,chemical,2015-01-29,754.00,2015
4,1.0000,Advanced Engine Technology,engine technology,2015-06-23,7330.92,2008
...,...,...,...,...,...,...
747,NaN,University West. Aus,research,NaT,3266.55,NaN
748,NaN,US DOE NNSA,research,NaT,5730.60,NaN
749,NaN,Valvoline Company,engine technology,NaT,1628.55,NaN
750,NaN,West Taylor,other,NaT,1000.00,NaN


In [59]:
# Renaming the columns 
Sales_df.columns = [['Exchange_Rate', 'Customer', 'Field', 'Date_of_Order', 'Total_Sales', 'Year_of_First_Order']]

In [60]:
# Replacing the empty values in the Exchange Rate column with 1. 
Sales_df['Exchange_Rate'].replace([None], [1]).tail()

747    1
748    1
749    1
750    1
751    1
Name: Exchange_Rate, dtype: float64

In [6]:
# Creating a new column in the data set, which puts all sales amounts in Canadian dollars. 
Sales_df['Sales_in_CAD'] = Sales_df['Exchange_Rate'] * Sales_df['Total_Sales'] 
Sales_df.head()

,Exchange_Rate,Customer,Field,Date_of_Order,Total_Sales,Year_of_First_Order,Sales_in_CAD
0,1.2563,3M Maplewood,solutions provider,2015-04-13,3267.00,2009,4104.3321
1,1.2188,ABB Australia Pty Limited,engine technology,2015-05-21,480.00,2008,585.0240
2,1.1800,Accutech Instrumentation,distributor,2015-01-13,18990.40,2013,22408.6720
3,1.2524,Advanced Analytical Technologies,chemical,2015-01-29,754.00,2015,944.3096
4,1.0000,Advanced Engine Technology,engine technology,2015-06-23,7330.92,2008,7330.9200


# Resampling data to make a plot that compares cost of ads to sales

In [7]:
# Grabbing two columns from the dataset and setting the index to Date of order.
Resampled_ind = Sales_df[['Date_of_Order', 'Total_Sales']].set_index('Date_of_Order')
Resampled_ind.head()

,Total_Sales
Date_of_Order,
2015-04-13,3267.00
2015-05-21,480.00
2015-01-13,18990.40
2015-01-29,754.00
2015-06-23,7330.92


In [8]:
# Data will be downsampled to month, takin the sum of the Sales in each month. 
Index_by_Month = Resampled_ind.resample('M', how=('sum'))   
Index_by_Month.head()

,Total_Sales
Date_of_Order,
2010-08-31,131518.52
2010-09-30,97850.61
2010-10-31,26195.94
2010-11-30,37621.75
2010-12-31,49579.74


In [9]:
# Deleting all rows before 2013-08-31. Keeping only two business years 2013 - 2015. 
Month_ind = Index_by_Month.drop(Index_by_Month.index[:36])
Month_ind.head()

,Total_Sales
Date_of_Order,
2013-08-31,78196.70
2013-09-30,55701.32
2013-10-31,106408.05
2013-11-30,98597.66
2013-12-31,77714.56


# Importing a new dataset (Cost of Ads per month)

In [48]:
#Loading another dataset from excel into python from a different sheet, and making the Date column the index.
Ads_df = pd.read_excel('Sales Data.xlsm', sheetname = 'Ads', index_col='Date')
Ads_df.head()

,Ad Cost
Date,
2013-08-01,947.94
2013-09-01,936.06
2013-10-01,935.72
2013-11-01,930.47
2013-12-01,948.00


In [51]:
# Renaming the column
# Resampling the date column, so that the month is synonymous with the other table 
Ads_df.columns = ['Ad_Cost']
Ads_df = Ads_df.resample('M', how=('sum'))   
Ads_df.head()

,Ad_Cost
Date,
2013-08-31,947.94
2013-09-30,936.06
2013-10-31,935.72
2013-11-30,930.47
2013-12-31,948.00


In [53]:
Combined_data = pd.concat([Month_ind, Ads_df], axis=1, join_axes=[Month_ind.index])
Combined_data.head()

,Total_Sales,Ad_Cost
Date_of_Order,,
2013-08-31,78196.70,947.94
2013-09-30,55701.32,936.06
2013-10-31,106408.05,935.72
2013-11-30,98597.66,930.47
2013-12-31,77714.56,948.00
